In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [ ]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/NSQIP-PLF/plf_combined.csv", index_col=0, na_values = -99)
data.head()

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Check data shape.

data.shape

In [ ]:
#Define variables of interest (predictor variables, inclusion/exclusion criteria, outcomes of interest).

variables = ['SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'CPT', 'INOUT', 'TRANST', 'AGE', 'DISCHDEST', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'HEIGHT', 'WEIGHT', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'OPTIME', 'TOTHLOS', 'HTOODAY', 'NSUPINFEC', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NOUPNEUMO', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NURNINFEC', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK', 'PODIAG', 'PODIAG10', 'STILLINHOSP', 'READMISSION1']

In [ ]:
#Remove unwanted columns and check data shape.

data = data[variables]

data.shape

In [ ]:
#See the ICD codes' unique value counts for the patient cohort.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()
icd_codes_index = icd_codes_df.index.tolist()

In [ ]:
#Exclude patients with ICD codes that were used less than 10 in the patient population.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()

icd_codes_df.columns =['Value']

icd_codes_df = icd_codes_df[icd_codes_df['Value'] >= 10]

icd_to_include = icd_codes_df.index.tolist()

data = data[data.PODIAG10.isin(icd_to_include)]

In [ ]:
#Get the descriptions for ICD codes.

icd10 = pd.read_csv("/content/drive/MyDrive/NSQIP-PLF/icd10_descriptions.csv", index_col = 'PODIAG10', encoding = 'latin1', low_memory = False)
icd10 = icd10.filter(items = icd_to_include, axis=0)

In [ ]:
#Save the ICD codes with descriptions and value counts.

icd = pd.concat([icd10, icd_codes_df], axis=1)
icd.to_csv('/content/drive/MyDrive/NSQIP-PLF/icd.csv')

In [ ]:
#Review the ICD table for excluding patients with exclude patients diagnosed with a fracture, neoplasm, infection, instrumentation related complications or lumbar/thoracic/sacral site diagnoses.

icd_to_exclude = ['M41.9' , 'T84.226A' , 'T84.216A' , 'C79.51' , 'T84.296A' , 'M71.38' , 'M46.26' , 'M48.56XA' , 'T84.84XA' , 'M84.58XA' , 'M41.85' , 'M41.26' , 'M41.56' , 'S32.009K' , 'Z98.1' , 'T84.498A' , 'M84.48XA' , 'S32.011A' , 'M41.25' , 'T84.038A' , 'G06.1' , 'M80.08XA' , 'T84.89XA' , 'S32.012A' , 'M40.205' , 'M41.87' , 'S32.021A' , 'S22.089A' , 'S32.041A' , 'S32.022A' , 'M80.88XA' , 'M41.80' , 'S32.059A' , 'M41.50' , 'S32.031A' , 'M96.3' , 'S32.010A' , 'S32.039A' , 'M40.209' , 'M41.35' , 'S32.032A' , 'M46.27' , 'M48.54XA' , 'M40.295' , 'S32.009A' , 'M71.30' , 'S32.042A' , 'S32.001A' , 'T84.63XA' , 'C79.49' , 'S22.080A' , 'S32.029A' , 'C41.2' , 'S32.048A' , 'T84.418A' , 'T84.098A' , 'M48.57XA' , 'M41.125' , 'T85.698A' , 'G06.2' , 'S32.058A' , 'S32.030A' , 'S22.088A' , 'S22.081A' , 'S32.018A' , 'M40.05' , 'M41.55' , 'T84.428A' , 'S32.10XA' , 'M12.88' , 'M41' , 'M96.0' , 'M41.86' , 'S32.019A' , 'S32.049A']

data = data[~data.PODIAG10.isin(icd_to_exclude)]

In [ ]:
#Drop patients with missing ICD codes.

data = data[data['PODIAG10'].notna()]

data.shape

In [ ]:
#Check data for ICD codes.

data['PODIAG10'].value_counts(normalize=False, dropna=False)

In [ ]:
#Define the CPT to codes to exclude (anterior cervical procedures, thoracic and/or lumbar fusion, revision, intraspinal lesion).

data.loc[data['CPT'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT1'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT2'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT3'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT4'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT5'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT6'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT7'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT8'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT9'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT10'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT1'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT2'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT3'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT4'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT5'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT6'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT7'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT8'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT9'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT10'] == 22800, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22802, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22804, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22808, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22810, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22812, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22862, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22865, 'CPT_EX'] = 'Yes'

data.loc[data['CPT_EX'] != 'Yes', 'CPT_EX'] = 'No'

data['CPT_EX'].value_counts(dropna=False)

In [ ]:
#Exclude patients with CPT codes that were assigned to be excluding.

data = data[(data['CPT_EX'] == 'No')]

data['CPT_EX'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for elective surgeries.

data['ELECTSURG'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for elective surgeries.

data = data[(data['ELECTSURG'] == 'Yes')]

data['ELECTSURG'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for emergency surgery.

data['EMERGNCY'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for emergency surgery.

data = data[(data['EMERGNCY'] == 'No')]

data['EMERGNCY'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for anesthesia type.

data['ANESTHES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for general anesthesia.

data = data[(data['ANESTHES'] == 'General')]

data['ANESTHES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for surgical specialties.

data['SURGSPEC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for surgical specialties.

data = data[(data['SURGSPEC'] == 'Neurosurgery') | (data['SURGSPEC'] == 'Orthopedics')]

data['SURGSPEC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for wound class.

data['WNDCLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for wound class.

data = data[(data['WNDCLAS'] == '1-Clean')]

data['WNDCLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for preoperative sepsis.

data['PRSEPIS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for preoperative sepsis.

data = data[(data['PRSEPIS'] == 'None')]

data['PRSEPIS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for ASA Class.

data['ASACLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for ASA class.

data = data[(data['ASACLAS'] != '4-Life Threat') & (data['ASACLAS'] != '5-Moribund') & (data['ASACLAS'] != 'None assigned')]

data['ASACLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Create BMI column.

lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)
print(min(data['BMI']))
print(max(data['BMI']))

In [ ]:
#Check data for race.

data['RACE_NEW'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for ethnicity.

data['ETHNICITY_HISPANIC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify race and ethnicity columns.

data.loc[data['RACE_NEW'] == 'White', 'RACE'] = 'White'
data.loc[data['RACE_NEW'] == 'Black or African American', 'RACE'] = 'Black or African American'
data.loc[data['RACE_NEW'] == 'Asian', 'RACE'] = 'Asian'
data.loc[data['RACE_NEW'] == 'American Indian or Alaska Native', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Other Pacific Islander', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Pacific Islander', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Some Other Race', 'RACE'] = 'Other'

data.loc[data['ETHNICITY_HISPANIC'] == 'Yes', 'RACE'] = 'Hispanic'

data['RACE'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for transfer status.

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify transfer status column.

data.loc[data['TRANST'] == 'Not transferred (admitted from home)', 'TRANST'] = 'Not transferred'
data.loc[data['TRANST'] == 'Nursing home - Chronic care - Intermediate care', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'From acute care hospital inpatient', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Outside emergency department', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Transfer from other', 'TRANST'] = 'Transferred'

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for dyspnea.

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify dyspnea column.

data.loc[data['DYSPNEA'] == 'No', 'DYSPNEA'] = 'No'
data.loc[data['DYSPNEA'] == 'MODERATE EXERTION', 'DYSPNEA'] = 'Yes'
data.loc[data['DYSPNEA'] == 'AT REST', 'DYSPNEA'] = 'Yes'

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for diabetes status.

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify diabetes column.

data.loc[data['DIABETES'] == 'NO', 'DIABETES'] = 'No'
data.loc[data['DIABETES'] == 'NON-INSULIN', 'DIABETES'] = 'Yes'
data.loc[data['DIABETES'] == 'INSULIN', 'DIABETES'] = 'Yes'

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Cast ASA class as ordered categorical.

cat_type1 = CategoricalDtype(categories=['1-No Disturb','2-Mild Disturb','3-Severe Disturb'], ordered=True)
data['ASACLAS'].astype(cat_type1)

In [ ]:
#Cast functional status as ordered categorical.

cat_type2 = CategoricalDtype(categories=['Unknown','Independent','Partiallly Dependent', 'Totally Dependent'], ordered=True)
data['FNSTATUS2'].astype(cat_type2)

In [ ]:
#Convert 90+ to 91 and AGE column to integer.

data.loc[data['AGE'] == '90+', 'AGE'] = 91
data['AGE'] = pd.to_numeric(data['AGE'], downcast='integer')

In [ ]:
#Show patients for each CPT code.

data['CPT'].value_counts()

In [ ]:
#Classify operations into single- vs. multiple-levels and create a column named 'LEVELS' for it.

data.loc[data['CPT'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT1'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT2'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT3'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT4'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT5'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT6'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT7'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT9'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT10'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT1'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT2'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT3'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT4'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT5'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT6'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT7'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT9'] == 22614, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT10'] == 22614, 'LEVELS'] = 'Multiple'

data.loc[data['CPT'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT1'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT2'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT3'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT4'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT5'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT6'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT7'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT9'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT10'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT1'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT2'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT3'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT4'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT5'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT6'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT7'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT9'] == 22632, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT10'] == 22632, 'LEVELS'] = 'Multiple'

data.loc[data['CPT'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT1'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT2'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT3'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT4'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT5'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT6'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT7'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT9'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['CONCPT10'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT1'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT2'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT3'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT4'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT5'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT6'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT7'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT9'] == 22634, 'LEVELS'] = 'Multiple'
data.loc[data['OTHERCPT10'] == 22634, 'LEVELS'] = 'Multiple'

data.loc[data['LEVELS'] != 'Multiple', 'LEVELS'] = 'Single'

data['LEVELS'].value_counts(dropna=False)

In [ ]:
#See LOS per patient.

data['TOTHLOS'].value_counts()

In [ ]:
#Drop patients with unknown LOS.

data = data[data['TOTHLOS'].notna()]

In [ ]:
#Show LOS after dropping patients with unknown LOS.

data['TOTHLOS'].value_counts(dropna=False)

In [ ]:
#See 75th percentile of LOS.
data.TOTHLOS.quantile(0.75)

In [ ]:
#Convert total length of stay into categorical data in a column named 'LOS'.

data.loc[data['TOTHLOS'] <= data.TOTHLOS.quantile(0.75), 'LOS'] = 'No'
data.loc[data['TOTHLOS'] > data.TOTHLOS.quantile(0.75), 'LOS'] = 'Yes'

In [ ]:
#Show prolonged LOS as categorical.

data['LOS'].value_counts(dropna=False)

In [ ]:
#Define major complications.

data['MAJRCOMP'] = data['NWNDINFD'] + data['NORGSPCSSI'] + data['NDEHIS'] + data['NREINTUB'] + data['NPULEMBOL'] + data['NFAILWEAN'] + data['NRENAINSF'] + data['NOPRENAFL'] + data['NCNSCVA'] + data['NCDARREST'] + data['NCDMI'] + data['NOTHBLEED'] + data['NOTHDVT'] + data['NOTHSYSEP'] + data['NOTHSESHOCK']

In [ ]:
#Show number of major complications per patient.

data['MAJRCOMP'].value_counts()

In [ ]:
#Convert major complications into categorical data in a column named 'COMP'.

data.loc[data['MAJRCOMP'] == 0, 'COMP'] = 'No'
data.loc[data['MAJRCOMP'] >= 1, 'COMP'] = 'Yes'

In [ ]:
#Show major complications as categorical.

data['COMP'].value_counts()

In [ ]:
#Drop patients with unknown major complications status.

data = data[data['COMP'].notna()]

In [ ]:
#Show major complication status after dropping patients with unknown major complication status.

data['COMP'].value_counts(dropna=False)

In [ ]:
#Show readmission status.

data['READMISSION1'].value_counts(dropna=False)

In [ ]:
#Drop patients with unknown readmission status.

data = data[data['READMISSION1'].notna()]

In [ ]:
#Show readmission status after dropping patients with unknown readmission status.

data['READMISSION1'].value_counts(dropna=False)

In [ ]:
#Show discharge status.

data['DISCHDEST'].value_counts(dropna=False)

In [ ]:
#Convert discharge destination into binary data (home vs. non-home discharge) in a column named 'DISCHARGE'.

data.loc[data['DISCHDEST'] == 'Home', 'DISCHARGE'] = 'No'
data.loc[data['DISCHDEST'] == 'Facility Which was Home', 'DISCHARGE'] = 'No'
data.loc[data['DISCHDEST'] == 'Skilled Care, Not Home', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Rehab', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Separate Acute Care', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Multi-level Senior Community', 'DISCHARGE'] = 'Yes'

In [ ]:
#Show discharge destination status after converting it to binary data.

data['DISCHARGE'].value_counts(dropna=False)

In [ ]:
#Drop patients with unknown discharge status.

data = data[data['DISCHARGE'].notna()]

In [ ]:
#Show discharge status after dropping patients with unknown discharge status.

data['DISCHARGE'].value_counts(dropna=False)

In [ ]:
#Check data.

data.shape

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Drop unwanted columns.

drop = ['RACE_NEW', 'ETHNICITY_HISPANIC', 'CPT',  'DISCHDEST', 'ANESTHES', 'ELECTSURG', 'PRSEPIS', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'EMERGNCY', 'WNDCLAS', 'OPTIME', 'TOTHLOS', 'HTOODAY', 'NSUPINFEC', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NOUPNEUMO', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NURNINFEC', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK', 'PODIAG', 'PODIAG10', 'STILLINHOSP', 'CPT_EX', 'MAJRCOMP']
data.drop(drop, axis=1, inplace=True)

In [ ]:
#Save data.

data.to_csv('/content/drive/MyDrive/NSQIP-PLF/plf_clean.csv')

In [ ]:
#See categorical and continuous variables.

print('Numerical columns: {}'.format(list(data.select_dtypes('number').columns)))
print()
print('Categorical columns: {}'.format(list(data.select_dtypes('object').columns)))

In [ ]:
#Define numerical and categorical columns.

num_cols = ['AGE', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'BMI']

cat_cols = ['SEX', 'INOUT', 'TRANST', 'SURGSPEC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'ASACLAS', 'READMISSION1', 'RACE', 'LEVELS', 'COMP', 'LOS', 'DISCHARGE']

In [ ]:
#Check missing values for numerical columns.

missing_num = data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

missing_num = pd.DataFrame(missing_num)

missing_num.columns = ['Value']

missing_num = missing_num[missing_num['Value'] > 0]

print(missing_num.index)

missing_num = missing_num[missing_num['Value'] > 25]

missing_num = list(missing_num.index)

print(missing_num)

In [ ]:
#Drop numerical columns with missing values over 25%.

data.drop(missing_num, axis=1, inplace=True)

In [ ]:
#Define new numerical columns.

num_cols = [x for x in num_cols if x not in missing_num]
print(num_cols)

In [ ]:
#Impute missing numerical values.

num_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

In [ ]:
#Check numerical variables with missing values after imputation.

data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
#Check missing values for categorical columns.

missing_cat = data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

missing_cat = pd.DataFrame(missing_cat)

missing_cat.columns = ['Value']

missing_cat = missing_cat[missing_cat['Value'] > 0]

print(missing_cat.index)

missing_cat = missing_cat[missing_cat['Value'] > 25]

missing_cat = list(missing_cat.index)

print(missing_cat)

In [ ]:
#Drop categorical columns with missing values over 25%.

data.drop(missing_cat, axis=1, inplace=True)

In [ ]:
#Define new categorical columns.

cat_cols = [x for x in cat_cols if x not in missing_cat]
print(cat_cols)

In [ ]:
#Replace missing categorical values with 'Unknown'.

for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)

In [ ]:
#Check missing values after imputation.

data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
#Save imputed data.

data.to_csv('/content/drive/MyDrive/NSQIP-PLF/plf_imputed.csv')

In [ ]:
#Normalize data.

data[num_cols] = MinMaxScaler().fit_transform(data[num_cols])

In [ ]:
#Save scaled data.

data.to_csv('/content/drive/MyDrive/NSQIP-PLF/plf_scaled.csv')

In [ ]:
#One hot encoding for categorical values.

data_final = pd.get_dummies(data, columns = cat_cols, drop_first = True)

In [ ]:
#Save final data.

data_final.to_csv('/content/drive/MyDrive/NSQIP-PLF/plf_final.csv')